In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn.linear_model import LogisticRegression

In [18]:
#Load cleaned data
df = pd.read_csv('final_dataframe.csv')

In [19]:
#load committee data, downloaded from the FEC campaign finance website
committees = pd.read_csv('committees-2021-11-05T15_07_01.csv')
display(committees.head())
committees.party.unique()

,name,cycles,treasurer_name,committee_type,committee_type_full,filing_frequency,designation,designation_full,organization_type,organization_type_full,...,party,party_full,state,first_file_date,last_file_date,first_f1_date,last_f1_date,committee_id,candidate_ids,sponsor_candidate_ids
0,"ZIMMER BIOMET HOLDINGS, INC. POLITICAL ACTION ...","{2004,2006,2008,2010,2012,2014,2016,2018,2020,...","CERONE, CHRISTOPHER A. MR.",Q,PAC - Qualified,M,B,Lobbyist/Registrant PAC,C,Corporation,...,NaN,NaN,DC,2004-04-17,2021-11-02,2004-04-17,2021-03-16,C00399386,{},NaN
1,"UNITED HEALTH SERVICES PAC, INC.","{2004,2006,2008,2010,2012,2014,2016,2018,2020,...","STEPHENS, SARA RALSTON",Q,PAC - Qualified,Q,U,Unauthorized,C,Corporation,...,NaN,NaN,GA,2004-04-22,2021-07-30,2004-04-22,2019-08-30,C00400135,{},NaN
2,AMERISOURCEBERGEN CORPORATION ASSOCIATES PAC (...,"{2004,2006,2008,2010,2012,2014,2016,2018,2020,...","COOK, DOUG",Q,PAC - Qualified,M,B,Lobbyist/Registrant PAC,C,Corporation,...,NaN,NaN,PA,2004-05-12,2021-10-19,2004-05-12,2021-02-03,C00400929,{},NaN
3,UNITED DEMOCRATIC CAMPAIGN OF ALAMEDA COUNTY (...,"{2004,2006,2008,2010,2012,2014,2016,2018}","SANBORN, GREGORY E.",N,PAC - Nonqualified,T,U,Unauthorized,NaN,NaN,...,NaN,NaN,CA,2004-07-19,2017-01-31,2004-07-19,2014-04-12,C00403774,{},NaN
4,"ADVANCED MICRO DEVICES, INC. PAC","{2004,2006,2008,2010,2012,2014,2016,2018,2020,...","CROSBY, CALEB",N,PAC - Nonqualified,Q,B,Lobbyist/Registrant PAC,C,Corporation,...,NaN,NaN,VA,2004-08-06,2021-07-31,2004-08-06,2021-02-17,C00404483,{},NaN


array([nan, 'DEM', 'REP', 'IND', 'UN', 'LIB', 'GRE', 'OTH', 'GRN', 'WFP',
       'NPA', 'REF', '.', 'ACE', 'IDP', 'NNE', 'CON', 'W', 'N', 'AIP',
       'UNK', 'PRO', 'DFL', 'SEP', 'DIS', 'POP', 'UST', 'RAP', 'MAA',
       'CRV', 'PRC', 'NON', 'NAT', 'PPY', 'STA', 'COM', 'FED', 'WTP',
       'ICD', 'UNI', 'AUT', 'GOP', 'AMP', 'PFP', 'SED', 'GWP', 'SWP',
       'REC', 'LBL', 'WRI', 'IAP', 'SLP', 'PAF', 'LMN', 'HRP', 'PAC',
       '  0', 'WOR', 'FWP', 'NPP', 'SOC', 'S4A', 'USA', 'DCG', 'US',
       'CST', 'MNC', 'BDY', 'ALP', 'PBP', 'FSP', 'DNC', 'DGR', 'UKN',
       'AKI', 'WAR', 'NJC', 'AIC', 'ECL', 'TX', 'UC', 'PRU', 'OKL', 'VPA',
       'LBR', 'APF', 'UUP', 'TUP', 'CO', 'UPA', 'EMP', 'HSA', 'NAP',
       'VET', 'CNC', 'PPA', 'FRE', 'SUB', 'VOL', 'CMD', 'D/C', 'IGD',
       'FL', 'OH', 'DRP', 'MAR', 'CIT', 'UNA', 'LA', 'PC', 'WNC', 'SUS',
       'TN', 'MN', 'FLP', 'NSM', 'APC', 'PPT', 'LRU', 'SAM', 'PPP', 'TEX',
       'NLP', 'WAD', 'WF', 'PPO', 'MD', 'NP', 'PBM', 'R', 'WHG', 'J',
    

In [20]:
committees.loc[committees.name == 'TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE'].party

9106    NaN
Name: party, dtype: object

Trump campaign party is not Republican, change that

In [21]:
committees.loc[9106, 'party'] = 'REP'
print(committees.loc[committees.name == 'TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE'].party)

9106    REP
Name: party, dtype: object


In [22]:
#Filter committees to just Republican or Democrat
reps_or_dems = committees.loc[(committees['party'] == 'DEM') | (committees['party'] == 'REP')]
reps_or_dems = reps_or_dems.loc[:, ['name', 'party']]
reps_or_dems.head()

,name,party
7,DEMOCRATIC ALLIANCE FOR ACTION,DEM
12,FRIENDS OF JOHN THUNE,REP
16,CASTOR FOR CONGRESS,DEM
19,SPENCER FOR SENATE COMMITTEE,REP
28,FRIENDS OF DARYL CAMPBELL,DEM


In [23]:
reps_or_dems.rename(columns = {'name': 'Advertiser_Name'}, inplace = True)
#merge our data with committees 
party_df = df.merge(reps_or_dems, on = 'Advertiser_Name')
party_df.head()

,Ad_Type,Regions,Advertiser_Name,Date_Range_Start,Date_Range_End,Num_of_Days,Impressions,Spend_USD,Age_Targeting,Gender_Targeting,Geo_Targeting_Included,Geo_Targeting_Excluded,Spend_Range_Min_USD,Spend_Range_Max_USD,Areas_Targeted,Ad_Type_Code,Impression_Type,party
0,Text,US,VAN DREW FOR CONGRESS,2019-12-19,2019-12-20,2.0,≤ 10k,1k-50k,Not targeted,Not targeted,United States,Not targeted,1000.0,50000.0,{'United States'},2,1,REP
1,Text,US,VAN DREW FOR CONGRESS,2019-12-19,2019-12-20,2.0,10k-100k,1k-50k,Not targeted,Not targeted,United States,Not targeted,1000.0,50000.0,{'United States'},2,2,REP
2,Video,US,VAN DREW FOR CONGRESS,2020-10-22,2020-10-23,2.0,100k-1M,1k-50k,Not targeted,Not targeted,"NJ-2,New Jersey,United States",Not targeted,1000.0,50000.0,"{'NJ-2,New Jersey,United States'}",1,3,REP
3,Video,US,VAN DREW FOR CONGRESS,2020-10-23,2020-11-02,11.0,100k-1M,1k-50k,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65, Unknow...","Male, Female, Unknown gender","NJ-2,New Jersey,United States",Not targeted,1000.0,50000.0,"{'NJ-2,New Jersey,United States'}",1,3,REP
4,Video,US,VAN DREW FOR CONGRESS,2020-09-22,2020-10-21,30.0,1M-10M,1k-50k,Not targeted,Not targeted,"NJ-2,New Jersey,United States",Not targeted,1000.0,50000.0,"{'NJ-2,New Jersey,United States'}",1,4,REP


In [24]:
#reduce dataframe to just geotargeting and party
party_df = party_df[['Geo_Targeting_Included', 'party']]
print('Length all data: ', len(df))
print('Length two party data: ', len(party_df))

Length all data:  81228
Length two party data:  36429


Over 50% of advertisers in dataset are not politically affiliated to Democrat or Republican commitees from the FEC dataset

In [25]:
#Encode party
party_df['Party_Encoded'] = np.zeros(len(party_df))
for index, row_series in party_df.iterrows():
    if party_df.at[index, 'party'] == 'REP':
        party_df.at[index, 'Party_Encoded'] = 0
    else:
        party_df.at[index, 'Party_Encoded'] = 1

In [26]:
party_df.head()

,Geo_Targeting_Included,party,Party_Encoded
0,United States,REP,0.0
1,United States,REP,0.0
2,"NJ-2,New Jersey,United States",REP,0.0
3,"NJ-2,New Jersey,United States",REP,0.0
4,"NJ-2,New Jersey,United States",REP,0.0


In [27]:
party_df.to_csv('mid_party_df.csv', index = False, header=True)

In [28]:
#load zipcode data
zips =  pd.read_csv('free-zipcode-database-Primary.csv')
zips.head()

,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
0,705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,False,NaN,NaN,NaN
1,610,STANDARD,ANASCO,PR,PRIMARY,18.28,-67.14,NA-US-PR-ANASCO,False,NaN,NaN,NaN
2,611,PO BOX,ANGELES,PR,PRIMARY,18.28,-66.79,NA-US-PR-ANGELES,False,NaN,NaN,NaN
3,612,STANDARD,ARECIBO,PR,PRIMARY,18.45,-66.73,NA-US-PR-ARECIBO,False,NaN,NaN,NaN
4,601,STANDARD,ADJUNTAS,PR,PRIMARY,18.16,-66.72,NA-US-PR-ADJUNTAS,False,NaN,NaN,NaN


In [29]:
zips['State'].unique()

array(['PR', 'NJ', 'NY', 'AE', 'VI', 'MA', 'ME', 'NH', 'VT', 'CT', 'RI',
       'DE', 'PA', 'WV', 'KY', 'TN', 'VA', 'GA', 'IN', 'OH', 'IL', 'IA',
       'MN', 'WI', 'MT', 'ND', 'SD', 'KS', 'MO', 'NE', 'CO', 'WY', 'ID',
       'UT', 'AZ', 'NM', 'TX', 'CA', 'NV', 'OR', 'WA', 'AK', 'GU', 'HI',
       'AS', 'PW', 'FM', 'MP', 'MH', 'FL', 'SC', 'AL', 'MS', 'LA', 'AR',
       'OK', 'MI', 'DC', 'MD', 'NC', 'AA', 'AP'], dtype=object)

In [30]:
#dict of states and abbreviations
us_states = {
	'Alabama': 'AL',
	'Alaska': 'AK',
	'Arizona': 'AZ',
	'Arkansas': 'AR',
    'American Samoa': 'AS',
	'California': 'CA',
	'Colorado': 'CO',
	'Connecticut': 'CT',
	'Delaware': 'DE',
	'District of Columbia': 'DC',
	'Florida': 'FL',
	'Georgia': 'GA',
    'Guam': 'GU',
	'Hawaii': 'HI',
	'Idaho': 'ID',
	'Illinois': 'IL',
	'Indiana': 'IN',
	'Iowa': 'IA',
	'Kansas': 'KS',
	'Kentucky': 'KY',
	'Louisiana': 'LA',
	'Maine': 'ME',
	'Maryland': 'MD',
	'Massachusetts': 'MA',
	'Michigan': 'MI',
    'Minnesota': 'MN',
	'Mississippi': 'MS',
	'Missouri': 'MO',
	'Montana': 'MT',
	'Nebraska': 'NE',
	'Nevada': 'NV',
	'New Hampshire': 'NH',
	'New Jersey': 'NJ',
	'New Mexico': 'NM',
	'New York': 'NY',
	'North Carolina': 'NC',
	'North Dakota': 'ND',
	'Ohio': 'OH',
	'Oklahoma': 'OK',
	'Oregon': 'OR',
	'Pennsylvania': 'PA',
    'Puerto Rico' : 'PR',
	'Rhode Island': 'RI',
	'South Carolina': 'SC',
	'South Dakota': 'SD',
	'Tennessee': 'TN',
	'Texas': 'TX',
	'Utah': 'UT',
	'Vermont': 'VT',
	'Virginia': 'VA',
    'Virgin Islands': 'VI',
	'Washington': 'WA',
	'West Virginia': 'WV',
	'Wisconsin': 'WI',
	'Wyoming': 'WY'
}

In [31]:
#separate all values in the geo targeting column into a list
list_zips = [item.split(',') for item in party_df['Geo_Targeting_Included']]

In [ ]:
#list for new column
state_column = []
#loop through geographic targetings
for x in range(len(list_zips)):
    if x%5000 == 0:
        print(x)
    list_of_states = []
    for zip in list_zips[x]:
        if zip.isdigit():
            int_zip = int(zip)
            zips_item = zips.loc[zips['Zipcode'] == int_zip]
            if(len(zips_item) == 1):
                state = zips_item['State'].item()
                list_of_states.append(state)
        for s in us_states:
            if s in zip:
                list_of_states.append(us_states[s])
    list_of_states = set(list_of_states)
    state_column.append(list(list_of_states))

0
5000
10000
15000
20000
25000
30000


In [ ]:
#create state column
party_df['State'] = state_column
party_df['State'].sample(10)

In [ ]:
party_df.sample(10)

In [ ]:
#get dummy column for each state
dummy_states = pd.get_dummies(party_df['State'].explode()).sum(level=0)
dummy_states['Party'] = party_df['Party_Encoded']

In [ ]:
dummy_states.sample(10)

In [ ]:
dummy_states.to_csv('party_dataframe.csv', index = False, header=True)